<a href="https://colab.research.google.com/github/sofiagpmg/Detecting-Cyberbully/blob/main/Fine_Tuning_XLNet_Model_for_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tuning XLNet Model for Text Classification


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/content/sample_data/sample1.csv")
#df.set_index('index', inplace=True)

In [ ]:
df.head()

,PAT_ID,CIM_SUBCAT,TXN_CODE,TXN_DESCRIPTION,DIAGNOSIS_CODE
0,PAT110,Pharmacy-Anaesthetics (Local & General),MAAJJDIPRI1P20,Propofol-Lipuro Emulsion 1% 20 Ml Inj B [Ha],I61.0
1,PAT110,Pharmacy-Anaesthetics (Local & General),MAAJJLIGNOC1P5,Lignocaine Hcl 1% Inj 5Ml B (Ml),I61.0
2,PAT110,Pharmacy-Anaesthetics (Local & General),MAAJJMARCAADR.5P20,Marcaine (Bupivacaine) 0.5% + Adrenaline 5Mg/5...,I61.0
3,PAT110,Pharmacy-Anaesthetics (Local & General),MAAJPOXYNO10,Oxynorm 10Mg/Ml 1 Ml Inj (Oxycodone) Pp [Ha],I61.0
4,PAT110,Pharmacy-Anti-Infective,MAAJAAMICI500,Amicin 500Mg/2Ml Inj (Amikacin) B,I61.0


In [ ]:
df.DIAGNOSIS_CODE.value_counts()

J18.9    2860
A09.9    2628
J10.0    2473
J40      1926
K29.6    1823
         ... 
S13.6       2
I27.0       2
C81         1
T83.1       1
C92.1       1
Name: DIAGNOSIS_CODE, Length: 817, dtype: int64

In [ ]:
df.isna().shape

(80240, 5)

In [ ]:
df = df.dropna()  

In [ ]:
df1 =  df.groupby("PAT_ID")['TXN_DESCRIPTION'].apply(lambda tags: ','.join(tags)).reset_index()

In [ ]:
df2 = df.dropna(subset=['Doc']).groupby(['Topic'], as_index = False).agg({'Doc': lambda x: ' '.join(x.dropna())})

In [ ]:
df.dtypes

PAT_ID             object
CIM_SUBCAT         object
TXN_CODE           object
TXN_DESCRIPTION    object
DIAGNOSIS_CODE     object
dtype: object

In [ ]:
df1.head()

,PAT_ID,TXN_DESCRIPTION
0,PAT-1,"Desmanol Solution (Chg Entry Code),Alcohol Swa..."
1,PAT-10,"Desmanol Solution (Chg Entry Code),Sodium Chlo..."
2,PAT-100,Meronem 500Mg Inj (Meropenem Trihyrate) (Powde...
3,PAT-101,"Arcoxia 120Mg Tab (Etoricoxib) B,Ceftrex 1G In..."
4,PAT-102,"Propofol-Lipuro Emulsion 1% 20 Ml Inj B [Ha],..."


In [ ]:
df.head()

,PAT_ID,CIM_SUBCAT,TXN_CODE,TXN_DESCRIPTION,DIAGNOSIS_CODE
0,PAT110,Pharmacy-Anaesthetics (Local & General),MAAJJDIPRI1P20,Propofol-Lipuro Emulsion 1% 20 Ml Inj B [Ha],I61.0
1,PAT110,Pharmacy-Anaesthetics (Local & General),MAAJJLIGNOC1P5,Lignocaine Hcl 1% Inj 5Ml B (Ml),I61.0
2,PAT110,Pharmacy-Anaesthetics (Local & General),MAAJJMARCAADR.5P20,Marcaine (Bupivacaine) 0.5% + Adrenaline 5Mg/5...,I61.0
3,PAT110,Pharmacy-Anaesthetics (Local & General),MAAJPOXYNO10,Oxynorm 10Mg/Ml 1 Ml Inj (Oxycodone) Pp [Ha],I61.0
4,PAT110,Pharmacy-Anti-Infective,MAAJAAMICI500,Amicin 500Mg/2Ml Inj (Amikacin) B,I61.0


In [ ]:
df.sample(10)['Text'].tolist()

['@WithTrish @JRehling http://t.co/NKMsZ2wxkv',
 "RT @jurijuri: @freebsdgirl It's like Total Recall but with only two breasts. MAYBE.",
 '@MuslimTwo LOL. Yeah I can just see a Daesh betting at a Texas auction.  ROFL.',
 'SPOCK DIED???',
 "@TheDreamIsHead that's a markov bot. not a human.",
 '@KellyTurner99 @isra_jourisra @fei989 Yeah, there is no problem to be concerned about. http://t.co/68bVW2qYA6',
 "I'm getting too many friend reqs from people I don't know. I'd rather just manage a page and keep all that stuff separate from pics of leo.",
 'RT @Raqqa_Sl: #Exclusive: #Raqqa : 12 dead bodies belonging to #ISIS were found near Al-AQtan, no more details are available yet about the …',
 'i actually think the instant restaurants are the best part of the series #MKR',
 'RT @olivewhippet: Kat and Andre are serving "floaters" for dessert #MKR http://t.co/nhvsgYh3NF']

In [ ]:
df.dropna(subset=['oh_label'], inplace=True)

## Cleaning
 - Replace `#`
 - Remove username starting with `@`
 - Remove `links`

In [ ]:
! pip install tweet-preprocessor
import preprocessor as p

def clean_text(Text):
  Text = Text.replace("@","")
  return p.clean(Text)

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()

df['clean_text'] = df['Text'].astype(str).progress_map(clean_text)
#df_test['clean_text'] = df_test['text'].astype(str).progress_map(clean_text)

In [ ]:
# splitting the data into training and test dataset
X = df['clean_text']
y = df['oh_label']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
train_df = pd.DataFrame(X_train)
train_df['oh_label'] = y_train

eval_df = pd.DataFrame(X_test)
eval_df['oh_label'] = y_test

In [ ]:
train_df.shape, eval_df.shape

((10776, 2), (2695, 2))

In [ ]:
# transformers - SOTA implementation of pretrained models
!pip3 install -U simpletransformers 

Requirement already up-to-date: simpletransformers in /usr/local/lib/python3.6/dist-packages (0.51.9)


In [ ]:
df.dropna(subset=['oh_label'], inplace=True)

In [ ]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)
'''
args = {
   'output_dir': 'outputs/',
   'cache_dir': 'cache/',
   'fp16': True,
   'fp16_opt_level': 'O1',
   'max_seq_length': 256,
   'train_batch_size': 8,
   'eval_batch_size': 8,
   'gradient_accumulation_steps': 1,
   'num_train_epochs': 3,
   'weight_decay': 0,
   'learning_rate': 4e-5,
   'adam_epsilon': 1e-8,
   'warmup_ratio': 0.06,
   'warmup_steps': 0,
   'max_grad_norm': 1.0,
   'logging_steps': 50,
   'evaluate_during_training': False,
   'save_steps': 2000,
   'eval_all_checkpoints': True,
   'use_tensorboard': True,
   'overwrite_output_dir': True,
   'reprocess_input_data': False,
}

'''

# Create a ClassificationModel
model = ClassificationModel('xlnet', 'xlnet-base-cased', args={'num_train_epochs':4, 'train_batch_size':32, 'max_seq_length':128}) # You can set class weights by using the optional weight argument

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.f1_score)

INFO:filelock:Lock 139673219091816 acquired on /root/.cache/huggingface/transformers/06bdb0f5882dbb833618c81c3b4c996a0c79422fa2c95ffea3827f92fc2dba6b.da982e2e596ec73828dbae86525a1870e513bd63aae5a2dc773ccc840ac5c346.lock


INFO:filelock:Lock 139673219091816 released on /root/.cache/huggingface/transformers/06bdb0f5882dbb833618c81c3b4c996a0c79422fa2c95ffea3827f92fc2dba6b.da982e2e596ec73828dbae86525a1870e513bd63aae5a2dc773ccc840ac5c346.lock


INFO:filelock:Lock 139672221019328 acquired on /root/.cache/huggingface/transformers/9461853998373b0b2f8ef8011a13b62a2c5f540b2c535ef3ea46ed8a062b16a9.3e214f11a50e9e03eb47535b58522fc3cc11ac67c120a9450f6276de151af987.lock


INFO:filelock:Lock 139672221019328 released on /root/.cache/huggingface/transformers/9461853998373b0b2f8ef8011a13b62a2c5f540b2c535ef3ea46ed8a062b16a9.3e214f11a50e9e03eb47535b58522fc3cc11ac67c120a9450f6276de151af987.lock


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

...

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:391: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


INFO:simpletransformers.classification.classification_model: Training of xlnet model complete. Saved to outputs/.
/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:970: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.738888040613224, 'tp': 315, 'tn': 2200, 'fp': 101, 'fn': 79, 'acc': 0.7777777777777778, 'eval_loss': 0.2707264343025923}


In [ ]:
result

{'acc': 0.935064935064935,
 'eval_loss': 0.2536053733497702,
 'fn': 87,
 'fp': 88,
 'mcc': 0.7401659311892613,
 'tn': 2213,
 'tp': 307}

In [ ]:
accuracy_per_class(predictions, true_vals)

In [ ]:
predictions, raw_outputs = model.predict(df_test.clean_text.tolist())

In [ ]:
sample_sub=pd.read_csv("sample_submission.csv")
sample_sub['target'] = predictions

sample_sub.to_csv("sub.csv", index=False)